In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [ ]:
text=(open("../input/data-51/D51.txt").read())
text=text.lower()
text

In [ ]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [ ]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [ ]:
model = Sequential()
model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
print(model.summary())

In [ ]:
checkpoint = ModelCheckpoint("Savan_JOEY51.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

model.fit(X_modified, Y_modified, epochs=10, batch_size=50)
# model.save_weights('text_generator_JOEY.h5')
model.save_weights('Savan_JOEY51.h5')

In [ ]:
# model.save_weights('text_generator_JOEY13.h5')

In [ ]:
model.load_weights('../input/weights/text_generator_JOEY13.h5')

In [ ]:
import random 
T=random.randint(100, 1000)

string_mapped = X[T]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(500):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [ ]:
#combining text
txt=input()
# text1=print("=")
for char in full_string:

    txt = txt+char
txt